# Using SQLAlchemy and RDS from Jupyter

### Establish the DB Connection

In [1]:
import config

from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

#### Note: these are the credentials you create when you set up the RDS database

In [2]:
engine = create_engine(f"postgresql://{config.USER}:{config.PASSWORD}@{config.HOST}:{config.PORT}/{config.DATABASE}")

### Import the CSVs

In [3]:
import os
import pandas as pd

In [4]:
user_data_df = pd.read_csv(os.path.join('Resources', 'data', 'user_data.csv'))
user_payment_df = pd.read_csv(os.path.join('Resources', 'data', 'user_payment.csv'))

In [5]:
user_data_df.head()

,id,first_name,last_name,active_user,street_address,state,username
0,1,Cletus,Lithcow,False,78309 Riverside Way,Virginia,ibearham0
1,2,Caz,Felgat,False,83 Hazelcrest Place,Alabama,wwaller1
2,3,Kerri,Crowson,False,112 Eliot Pass,North Carolina,ichesnut2
3,4,Freddie,Caghy,False,15 Merchant Way,New York,tsnarr3
4,5,Sadella,Deuss,False,079 Acker Avenue,Tennessee,fwherrit4


In [6]:
user_payment_df.head()

,billing_id,username,cc_encrypted
0,1,ibearham0,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
1,2,wwaller1,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
2,3,ichesnut2,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
3,4,tsnarr3,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
4,5,fwherrit4,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87


In [7]:
joined_df = user_data_df.merge(user_payment_df, on='username', how='inner')
joined_df.head()

,id,first_name,last_name,active_user,street_address,state,username,billing_id,cc_encrypted
0,1,Cletus,Lithcow,False,78309 Riverside Way,Virginia,ibearham0,1,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
1,2,Caz,Felgat,False,83 Hazelcrest Place,Alabama,wwaller1,2,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
2,3,Kerri,Crowson,False,112 Eliot Pass,North Carolina,ichesnut2,3,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
3,4,Freddie,Caghy,False,15 Merchant Way,New York,tsnarr3,4,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
4,5,Sadella,Deuss,False,079 Acker Avenue,Tennessee,fwherrit4,5,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87


In [8]:
dropna_df = joined_df.dropna()
dropna_df.head()

,id,first_name,last_name,active_user,street_address,state,username,billing_id,cc_encrypted
0,1,Cletus,Lithcow,False,78309 Riverside Way,Virginia,ibearham0,1,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
1,2,Caz,Felgat,False,83 Hazelcrest Place,Alabama,wwaller1,2,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
2,3,Kerri,Crowson,False,112 Eliot Pass,North Carolina,ichesnut2,3,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
3,4,Freddie,Caghy,False,15 Merchant Way,New York,tsnarr3,4,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
4,5,Sadella,Deuss,False,079 Acker Avenue,Tennessee,fwherrit4,5,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87


In [9]:
cleaned_df = dropna_df.loc[dropna_df['active_user'] == True]
cleaned_df.head()

,id,first_name,last_name,active_user,street_address,state,username,billing_id,cc_encrypted
5,6,Fraser,Korneev,True,76084 Novick Court,Minnesota,fstappard5,6,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
6,7,Demott,Rapson,True,86320 Dahle Park,District of Columbia,lhambling6,7,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
14,15,Sadella,Jaram,True,7528 Waxwing Terrace,Connecticut,wheinerte,15,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
16,17,Hewitt,Trammel,True,2455 Corry Alley,North Carolina,droughsedgeg,17,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
18,19,Ted,Knowlys,True,31 South Drive,Ohio,ydudeniei,19,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87


In [10]:
clean_user_df = cleaned_df[['id', 'first_name', 'last_name', 'username']]
clean_user_df.head()

,id,first_name,last_name,username
5,6,Fraser,Korneev,fstappard5
6,7,Demott,Rapson,lhambling6
14,15,Sadella,Jaram,wheinerte
16,17,Hewitt,Trammel,droughsedgeg
18,19,Ted,Knowlys,ydudeniei


In [11]:
clean_billing_df = cleaned_df[['billing_id', 'street_address', 'state', 'username']]
clean_billing_df.head()

,billing_id,street_address,state,username
5,6,76084 Novick Court,Minnesota,fstappard5
6,7,86320 Dahle Park,District of Columbia,lhambling6
14,15,7528 Waxwing Terrace,Connecticut,wheinerte
16,17,2455 Corry Alley,North Carolina,droughsedgeg
18,19,31 South Drive,Ohio,ydudeniei


In [12]:
clean_payment_df = cleaned_df[['billing_id', 'cc_encrypted']]
clean_payment_df.head()

,billing_id,cc_encrypted
5,6,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
6,7,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
14,15,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
16,17,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87
18,19,a799fcafe47d7fb19bfb02cd83855fdfc34b9f87


### Write the Data To RDS Tables in the Cloud

In [13]:
try:
    clean_user_df.to_sql(
        'active_user',
        engine,
        if_exists='fail',
        chunksize=100,
        index=False
    )
except ValueError:
    print('Data already exists')

Data already exists


In [14]:
try:
    clean_billing_df.to_sql(
        'billing_info',
        engine,
        if_exists='fail',
        chunksize=100,
        index=False
    )
except ValueError:
    print('Data already exists')

Data already exists


In [15]:
try:
    clean_payment_df.to_sql(
        'payment_info',
        engine,
        if_exists='fail',
        chunksize=100,
        index=False
    )
except ValueError:
    print('Data already exists')

Data already exists


### Query the Database

In [16]:
sql_query = """
SELECT * from active_user;
"""

In [17]:
conn = engine.connect()
qr = conn.execute(sql_query)
query_result = [row for row in qr]
conn.close()

In [18]:
# Print the first ten results
query_result[:10]

[(6, 'Fraser', 'Korneev', 'fstappard5'),
 (7, 'Demott', 'Rapson', 'lhambling6'),
 (15, 'Sadella', 'Jaram', 'wheinerte'),
 (17, 'Hewitt', 'Trammel', 'droughsedgeg'),
 (19, 'Ted', 'Knowlys', 'ydudeniei'),
 (23, 'Annmarie', 'Lafond', 'fmyttonm'),
 (28, 'Toma', 'Sokell', 'bfletcherr'),
 (30, 'Ram', 'Lefever', 'gturleyt'),
 (31, 'Raddie', 'Heindle', 'calyukinu'),
 (33, 'Wallie', 'Caws', 'ckleinlererw')]

### Query the Database and Write the Result Directly to a DataFrame

In [19]:
query_df = pd.read_sql(sql_query, con=engine)
query_df.head(10)

,id,first_name,last_name,username
0,6,Fraser,Korneev,fstappard5
1,7,Demott,Rapson,lhambling6
2,15,Sadella,Jaram,wheinerte
3,17,Hewitt,Trammel,droughsedgeg
4,19,Ted,Knowlys,ydudeniei
5,23,Annmarie,Lafond,fmyttonm
6,28,Toma,Sokell,bfletcherr
7,30,Ram,Lefever,gturleyt
8,31,Raddie,Heindle,calyukinu
9,33,Wallie,Caws,ckleinlererw
